Cartar de posible desvio para la union

In [15]:
import sys
sys.path.append('..')

import pandas as pd
import collections
import os

from docxtpl import DocxTemplate

from config import RUTA_UNIDAD_ONE_DRIVE
from config import RUTA_LOCAL_ONE_DRIVE
from config import POSTGRES_UTEA

RUTA_COMPLETA = os.path.join(RUTA_UNIDAD_ONE_DRIVE, RUTA_LOCAL_ONE_DRIVE)

In [16]:
RUTA_COMPLETA

'G:\\OneDrive - Ingenio Azucarero Guabira S.A\\_DATOS_PYTHON'

In [17]:
path_xlsx_avance = RUTA_UNIDAD_ONE_DRIVE + r'\Ingenio Azucarero Guabira S.A\UTEA - SEMANAL - AVANCE COSECHA\2025\AVANCE SEMANAL\AVANCE DE COSECHA V1.xlsx'

In [18]:
xlsx_entregas = pd.read_excel(path_xlsx_avance, sheet_name='DETALLE ENTREGAS')
xlsx_avance = pd.read_excel(path_xlsx_avance, sheet_name = 'AVANCE')

In [19]:
xlsx_entregas = xlsx_entregas.drop(xlsx_entregas.index[-1])
xlsx_entregas = xlsx_entregas.drop(xlsx_entregas.index[-1])
xlsx_entregas['cod_cos'] = xlsx_entregas['cod_cos'].astype(int)
xlsx_entregas['cod_ca'] = xlsx_entregas['cod_ca'].astype(int)
xlsx_entregas['%ENTREGA'] = xlsx_entregas['%ENTREGA'] * 100
xlsx_entregas.head(3)

,inst,cod_cos,cod_ca,nom_ca,TOTAL COMPROMISO,TOTAL ENTREGAS,%ENTREGA
0,1,49,660,AGUILERA ANTELO ALEX ENRIQUE,982.61,627.21,63.831021
1,1,54,3361,ENRIQUEZ ROJAS ALBERTO,1615.39,1471.42,91.087601
2,1,55,3460,ESPINOZA MENDOZA SANTIAGO,5288.56,6223.57,117.679860


In [20]:
xlsx_avance.head(3)

,COD COS,INST,COD CA,NOMBRE,COMPROMISO,ENTREGAS,%ENTREGA,TOTAL AREA,COSECHADO,SIN COSECHAR,...,TCH,%DIFF AVANCE,TCH EST,PRODUC EST,EST PRODUC COSECHADO,DIF. EST ENTREGA,%DIF. EST ENTREGA,TN EN CAMPO,COMPROMISO RES,DIFERENCIA
0,1,50,515,AGUILERA OLGA RIVERO VDA DE,30572.11,27629.28,0.903741,618.28,457.44,154.87,...,60.399790,0.163882,57.639599,35292.15,26843.40,785.88,0.971556,8448.75,2942.83,5505.92
1,2,69,1180,BALCAZAR BALCAZAR JESUS E.,10965.29,11929.34,1.087918,200.94,198.31,0.57,...,60.155010,0.101007,53.082819,10569.32,10548.26,1381.08,0.884228,21.06,0.00,21.06
2,3,69,911,BALCAZAR JUSTINIANO MARIO,5422.67,3441.01,0.634560,122.97,64.95,57.37,...,52.979369,0.106382,51.427065,6289.53,3478.47,-37.46,1.010886,2811.06,1981.66,829.40


In [21]:
fecha_inf = '31 de octubre de 2025'
fecha_form = '31-10-2025'

fecha_img = ' de octubre de 2025'

cod_grupos = [152
]


In [22]:
def crear_reporte(lista_cod_cos):
    for cod in lista_cod_cos:
        grupo = xlsx_entregas[xlsx_entregas['cod_cos'] == cod].copy()
        
        # convertir datos del grupo a objetos
        list_obj_caneros = []
        for index, row in grupo.iterrows():
            canero = {
            'inst' : row['inst'],
            'cod_ca' : row['cod_ca'],
            'nom_ca' : row['nom_ca'],
            'compromiso' : row['TOTAL COMPROMISO'],
            'entrega' : row['TOTAL ENTREGAS'],
            'entrega_porcen' : row['%ENTREGA']
            }
            list_obj_caneros.append(collections.namedtuple("canero", canero.keys())(*canero.values()))
        # totales de los datos del grupo
        grupo_totales = {
            'compromiso' : grupo['TOTAL COMPROMISO'].sum(),
            'entregas' : grupo['TOTAL ENTREGAS'].sum(),
            'entrega_porcen' : grupo['TOTAL ENTREGAS'].sum() / grupo['TOTAL COMPROMISO'].sum() * 100
        }
        obj_grupo_totales = collections.namedtuple("grupo_totales", grupo_totales.keys())(*grupo_totales.values())
        
        # datos generales de avance de cosecha
        avance = xlsx_avance[xlsx_avance['COD COS'] ==cod]
        avance_totales = {
            'fecha_img' : fecha_img,
            'fecha_informe' : fecha_inf,
            'compromiso' : float(avance['COMPROMISO'].iloc[0]),
            'entrega' : float(avance['ENTREGAS'].iloc[0]),
            'entrega_porcen' : float(avance['%ENTREGA'].iloc[0]) * 100,
            'ha_canha' : float(avance['COSECHADO'].iloc[0] + avance['SIN COSECHAR'].iloc[0]),
            'ha_cosecha' : float(avance['COSECHADO'].iloc[0]),
            'cosecha_porcen' : float(avance['%COSECHADO'].iloc[0]) * 100,
            'tch' : float(avance['TCH'].iloc[0])
        }
        # datos generales a objeto
        obj_avance_totales = collections.namedtuple("avance_totales",avance_totales.keys())(*avance_totales.values())
        
        # generar reporte
        doc = DocxTemplate(RUTA_COMPLETA + '/templates/tpl_rpt_avance_posible_desvio.docx')
        context = {
            'insp' : list_obj_caneros,
            'grupo_totales' : obj_grupo_totales,
            'avance' : obj_avance_totales
        }
        doc.render(context)

        #REPROTE POSIBLE COMPRA DE CAÑA
        doc.save(RUTA_COMPLETA + '/informes/' + str(avance['COD CA'].iloc[0]) + '_RPDC_' + fecha_form + '_' + avance['NOMBRE'].iloc[0] + '.docx')
        print('Se generó reporte de grupo:', cod, ' - ', avance['NOMBRE'].iloc[0])

In [23]:
crear_reporte(cod_grupos)

Se generó reporte de grupo: 152  -  VARGAS SEGOVIA LUIS CASIMIRO
